In [1]:
import PIL.Image
from PIL import Image, ImageTk
from typing import Tuple
from tkinter import *
import cv2

In [2]:
%reload_ext ObjectTracker
%reload_ext PickerApp
from ObjectTracker import ObjectTracker
from PickerApp import PickerApp

stride = 6
margin = 30
scale_factor = 0.5
video = cv2.VideoCapture("videos/full_hd_man.mp4")

root = Tk()
app = PickerApp(root, video, scale_factor)
root.mainloop()

box = app.get_box()
print(box)

ObjectTracker(video, box, stride=stride, margin=margin, scale_factor=scale_factor, debug=True).track()

------------------
458 154
463 692
[308. 916. 468. 618.]
0.01600503921508789
0.01300358772277832
0.012002706527709961
0.012000083923339844
0.011003732681274414
0.011002063751220703
0.013003110885620117
0.012001514434814453
0.011003255844116211
0.011001825332641602
0.014002799987792969
0.01200103759765625
0.01200246810913086
0.01900482177734375
0.011002779006958008
0.01300191879272461
0.013002157211303711
0.010001420974731445
0.009002208709716797
0.010002613067626953
0.009994983673095703
0.011000394821166992
0.011000394821166992
0.010002851486206055
0.009001970291137695
0.011002779006958008
0.011002302169799805
0.011002302169799805
0.011002779006958008
0.012002229690551758
0.009001016616821289
0.01000213623046875
0.009002208709716797
0.009001493453979492
0.01600360870361328
0.010000944137573242
0.010002613067626953
0.012003183364868164
0.010001897811889648
0.009001016616821289
0.01000213623046875
0.010001659393310547
0.008002042770385742
0.010001182556152344
0.009001970291137695
0.01000